In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np


# Convert images to live image landmarks version:
Below takes in an array where each element is the file name of an image in the folder data/unlabled/. The script reads the image, runs a hand detection algorithm, then overlays the results and saves the output to the folder data/labeled/ with the original file name + "_marked.png"


TODO: Save the keypoints into a separate csv to create a training dataset

In [19]:
# For static images:
IMAGE_FILES = ["hands.jpg"]
with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
  for idx, file_ori in enumerate(IMAGE_FILES):
    file = "./data/unlabeled/" + file_ori
    print("iteration",idx,"file,",file)
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).

    # image = cv2.flip(cv2.imread(file), 1)
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the BGR image to RGB before processing.
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:

      for hand_landmarks in results.multi_hand_landmarks:
        # landmarks.append()
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite("./data/labeled/" +str(file_ori)[:-4]+ "marked.png",cv2.flip(image, 1))

iteration 0 file, ./data/unlabeled/hands.jpg


Now, we want the relative normalized coordinates of thes hand so it is invariant to location and image size!

In [5]:

# For static images:
IMAGE_FILES = ["hands.jpg"]
with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
  for idx, file_ori in enumerate(IMAGE_FILES):
    file = "./data/unlabeled/" + file_ori
    print("iteration",idx,"file,",file)
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).

    # image = cv2.flip(cv2.imread(file), 1)
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert the BGR image to RGB before processing.
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # Extract the hand landmarks
    landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the bounding box of the hand
            x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')
            for landmark in hand_landmarks.landmark:
                x, y = landmark.x, landmark.y
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
            
            # Normalize the coordinates with respect to the bounding box of the hand
            for landmark in hand_landmarks.landmark:
                x, y, z = landmark.x, landmark.y, landmark.z
                x_norm = (x - x_min) / (x_max - x_min)
                y_norm = (y - y_min) / (y_max - y_min)
                landmarks.append([x_norm, y_norm, z])

    # Convert the landmarks to a feature vector
    feature_vector = np.array(landmarks).flatten()

    # Print the feature vector
    print(feature_vector)

iteration 0 file, ./data/unlabeled/hands.jpg
[ 1.00000000e+00  4.34967835e-01 -3.47916576e-08  8.82582635e-01
  7.59323158e-01 -1.83860995e-02  6.89516073e-01  9.33540385e-01
 -2.44351812e-02  5.23791267e-01  9.92139709e-01 -3.15571316e-02
  3.82077427e-01  1.00000000e+00 -3.75392064e-02  5.04185301e-01
  6.06368761e-01  6.72204792e-03  3.23579050e-01  7.12554842e-01
 -1.75520517e-02  2.25997013e-01  8.39138381e-01 -4.18161079e-02
  1.51806618e-01  9.76945895e-01 -5.76517135e-02  4.77653627e-01
  4.08366269e-01  7.47234328e-04  2.59541127e-01  4.45197047e-01
 -1.72595009e-02  1.14985974e-01  5.40388545e-01 -3.80484127e-02
  0.00000000e+00  6.53343661e-01 -5.24306186e-02  4.98473873e-01
  2.32378382e-01 -1.09612327e-02  2.92032793e-01  2.19396905e-01
 -2.99397297e-02  1.55154329e-01  2.84264374e-01 -4.68715392e-02
  4.01836178e-02  3.75865428e-01 -5.75676374e-02  5.54889918e-01
  6.97741740e-02 -2.51207817e-02  3.97184439e-01  0.00000000e+00
 -4.07977998e-02  2.82935436e-01  1.45630962e